In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install pyserini==0.22.0
!pip install faiss-cpu

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [3]:
import json
import os
from bs4 import BeautifulSoup
from pyserini.search.lucene import LuceneSearcher

In [4]:
searcher = LuceneSearcher('/content/drive/MyDrive/indexes/sample_collection_jsonl')
searcher.set_language('fa')
hits = searcher.search('لات')

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

 1 s53.19 4.84050


dpr dataset creation

In [17]:
with open('/content/drive/MyDrive/single_verse.jsonl', 'r') as f:
  dict_data_list = []
  for line in f:
   data = json.loads(line)
   dict_data_list.append(data)

with open('/content/drive/MyDrive/makarem.xml', 'r', encoding="utf8") as f:
  content = f.read()
soup= BeautifulSoup(content, 'xml')

searcher = LuceneSearcher('/content/drive/MyDrive/indexes/sample_collection_jsonl')
searcher.set_language('fa')

dpr_data_list = []

for qa in dict_data_list:
  question = qa['question']
  dpr_data = {}
  dpr_data['dataset'] = 'Quran_QA'
  dpr_data['question'] = qa['question']
  dpr_data['answers'] = [qa['answers']]
  dpr_data['positive_ctxs'] = [{'title': '' , 'text': qa['context'],
                                'score': 1000, 'title_score':1, 'passage_id':qa['pq_id']}]
  dpr_data['negative_ctxs'] = []

  hits = searcher.search(question)
  hard_negatives = []
  # if len(hits)>5:
  #   negatives_number = 5
  # else:
  #   negatives_number = len(hits)

  for i in range(len(hits)):
    verse_data = {}
    # print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    verse_id = hits[i].docid
    verse_content = soup.find(id=verse_id).contents[0]
    if qa['answers'] not in verse_content:
      verse_data['passage_id'] = verse_id
      verse_data['text'] = verse_content
      verse_data['title'] = ''
      verse_data['score'] = hits[i].score
      verse_data['title_score'] = 0
      hard_negatives.append(verse_data)

    if len(hard_negatives) == 5:
      break

  dpr_data['hard_negative_ctxs'] = hard_negatives

  dpr_data_list.append(dpr_data)

  del dpr_data, hard_negatives, hits

In [18]:
len(dpr_data_list)

2471

In [19]:
import math
split_per = math.trunc(0.8 * len(dpr_data_list))

with open('/content/drive/MyDrive/DPR_train.json', 'w') as fout:
  json.dump(dpr_data_list[:split_per], fout)

with open('/content/drive/MyDrive/DPR_dev.json', 'w') as fout:
  json.dump(dpr_data_list[split_per:], fout)

In [20]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))


output_path = '/content/drive/MyDrive/DPR_dataset.jsonl'
dump_jsonl(dpr_data_list, output_path)

Wrote 2471 records to /content/drive/MyDrive/DPR_dataset.jsonl
